In [5]:
!pip install pyDOE
!pip install cma


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F9 '''
def F9(X):
    f = bn.F9()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 26.915 ,random_state=0 , l1_ratio=0.9898, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 69.69 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_9_2000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_9_400Samples.csv').iloc[:,-1]
path = "train_latent_100D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_100D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [10]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=837134, Mon Apr 13 14:00:20 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 5.143814275196570e+06 1.0e+00 2.54e-01  3e-01  3e-01 0:00.0
    2     30 5.143814001114732e+06 1.0e+00 2.45e-01  2e-01  2e-01 0:00.0
    3     45 5.143813627962054e+06 1.1e+00 2.39e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5143815.23750436] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5143814.28446466] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -3.738601594066164e+05 1.4e+00 4.26e-02  4e-02  4e-02 0:02.0
  200   3000 -8.932654380196305e+05 1.5e+00 2.09e-03  2e-03  2e-03 0:03.9
  300   4500 -8.946112246213527e+05 1.5e+00 1.21e-04  1e-04  1e-04 0:05.9
  400   6000 -8.946154505731296e+05 1.6e+00 6.23e-06  5e-06  6e-06 0:07.8
  500   7500 -8.946154611822991e+05 1.6e+00 3.96e-07  3e-07  4e-07 0:09.7
  600   9000 -8.946154612173671e+05 1.6e+00 2.20e-08  2e-08  2e-08 0:11.7
  700  10500 -8.946154612174863e+05 1.7e+00 2.15e-09  2e-09  2e-09 0:13.6
  800  12000 -8.946154612174872e+05 1.8e+00 1.28e-09  9e-10  1e-09 0:15.5
  900  13500 -8.946154612174882e+05 2.0e+00 9.95e-10  7e-10  9e-10 0:17.4
 1000  15000 -8.946154612174882e+05 2.2e+00 7.06e-10  5e-10  6e-10 0:19.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -8.946154612174882e+05 2.4e+00 6.79e-10  5e-10  6e-10 0:21.3
 1200  18000 -8.946154612174882e+05 2.5e+00 3.38e-10  2e-10  3e-10 0:23.2
 1300  19500 -8.946154612174882e+05 2.9e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-894615.461217, sigma=1.89e-10).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=2410)
  ')')


 2500  37500 -8.946154612174882e+05 5.6e+00 1.66e-10  9e-11  2e-10 0:48.1
 2600  39000 -8.946154612174882e+05 5.8e+00 1.43e-10  8e-11  1e-10 0:50.1
 2635  39525 -8.946154612174882e+05 5.9e+00 1.02e-10  6e-11  9e-11 0:50.7
termination on tolstagnation=2457 (Mon Apr 13 14:01:19 2020)
final/bestever f-value = -8.946155e+05 -8.946155e+05
incumbent solution: [-0.00208566  0.1000704  -0.02231641 -0.04141178  0.03219169 -0.0027093
  0.05565299  0.01335943 ...]
std deviations: [7.03963675e-11 7.21108242e-11 8.39185624e-11 8.53096603e-11
 6.72686984e-11 6.48115104e-11 7.10761852e-11 6.87626167e-11 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=946238, Mon Apr 13 14:01:19 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 4.971084541771214e+06 1.0e+00 2.53e-01  3e-01  3e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 14:01:19 2020)
final/bestever f-value = 4.971085e+06 4.971085e+06
incumbent solution: [-0.12913827  0.83006899  0.05295863

...

  100   1500 -5.665812352508595e+05 1.4e+00 2.99e-02  3e-02  3e-02 0:01.9
  200   3000 -8.939142126662219e+05 1.5e+00 1.34e-03  1e-03  1e-03 0:03.8
  300   4500 -8.946130582850100e+05 1.5e+00 9.37e-05  8e-05  9e-05 0:05.7
  400   6000 -8.946154510152340e+05 1.6e+00 5.98e-06  5e-06  6e-06 0:07.7
  500   7500 -8.946154611757724e+05 1.6e+00 3.73e-07  3e-07  3e-07 0:09.6
  600   9000 -8.946154612173773e+05 1.6e+00 2.32e-08  2e-08  2e-08 0:11.6
  700  10500 -8.946154612174863e+05 1.7e+00 2.49e-09  2e-09  2e-09 0:13.5
  800  12000 -8.946154612174863e+05 1.9e+00 1.67e-09  1e-09  1e-09 0:15.5
  900  13500 -8.946154612174872e+05 2.0e+00 1.11e-09  8e-10  9e-10 0:17.5
 1000  15000 -8.946154612174891e+05 2.3e+00 1.14e-09  8e-10  1e-09 0:19.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -8.946154612174882e+05 2.4e+00 8.51e-10  6e-10  7e-10 0:21.4
 1200  18000 -8.946154612174882e+05 2.6e+00 7.56e-10  5e-10  7e-10 0:23.4
 1300  19500 -8.946154612174882e+05 2.8e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-894615.461217, sigma=7.30e-10).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1299)
  ')')


 1400  21000 -8.946154612174882e+05 2.9e+00 6.46e-10  4e-10  6e-10 0:27.3
 1500  22500 -8.946154612174882e+05 3.0e+00 5.96e-10  4e-10  5e-10 0:29.2
 1600  24000 -8.946154612174882e+05 3.2e+00 3.34e-10  2e-10  3e-10 0:31.1
 1700  25500 -8.946154612174882e+05 3.5e+00 2.43e-10  2e-10  2e-10 0:33.1
 1800  27000 -8.946154612174882e+05 3.6e+00 2.26e-10  1e-10  2e-10 0:35.0
 1900  28500 -8.946154612174872e+05 3.8e+00 2.31e-10  1e-10  2e-10 0:37.0
 2000  30000 -8.946154612174882e+05 4.0e+00 2.30e-10  1e-10  2e-10 0:38.9
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 2100  31500 -8.946154612174882e+05 4.2e+00 1.77e-10  1e-10  2e-10 0:40.8
 2200  33000 -8.946154612174872e+05 4.5e+00 1.29e-10  8e-11  1e-10 0:42.8
 2300  34500 -8.946154612174882e+05 4.5e+00 1.13e-10  7e-11  1e-10 0:44.8
 2400  36000 -8.946154612174900e+05 4.6e+00 8.11e-11  5e-11  7e-11 0:46.7
 2500  37500 -8.946154612174882e+05 4.7e+00 6.65e-11  4e-11  6e-11 0:48.6
 2600  39000 -8.946154612174882e+05 4.9e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4971084.54177121] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=4971084.541771, sigma=2.57e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=4971084.541771, sigma=2.57e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=

  100   1500 -8.228548886933237e+10 1.8e+00 9.08e+01  9e+01  1e+02 0:01.9
  118   1770 -9.304418192185813e+11 1.8e+00 2.81e+02  3e+02  3e+02 0:02.2
termination on tolfacupx=1000.0 (Mon Apr 13 14:32:51 2020)
final/bestever f-value = -9.081992e+11 -9.304418e+11
incumbent solution: [  -55.85375281   531.2404111  -1125.11848772  -405.30826667
  2255.4912757   -320.82035284    45.39524409  -615.42079426 ...]
std deviations: [274.34990347 270.31458741 274.04015116 277.59546186 284.34118204
 272.46139223 276.84119496 267.87623477 ...]


In [11]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F9_100_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F9(X_Values.iloc[i,:50])
    SVM_Fun [i] = F9(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F9(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([-1.25286047]), count=array([1])) 5134.733686840241
SVM
ModeResult(mode=array([3082.44639436]), count=array([1])) 1707.2029381813634
ELN
ModeResult(mode=array([3.28862557e+15]), count=array([1])) 3792592861908371.5
